# Egobox

## Installation

You can install with the `pip` command as follows: 

```bash
pip install egobox
```

## Import

In [1]:
import numpy as np
import egobox as egx

# To display optimization information (none by default)
import logging
logging.basicConfig(level=logging.INFO)

## Continuous test functions 

In [2]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [3]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


## Ego optimizer written in Rust (a.k.a Egor)

In [4]:
egor = egx.Egor(g24, xspecs_g24, 
                     n_doe=10, 
                     n_cstr=n_cstr_g24, 
                     cstr_tol=1e-3,
                     infill_strategy=egx.InfillStrategy.WB2,
                     # expected=egx.ExpectedOptimum(val=-5.50, tol=1e-2),
                     # outdir="./out",
                     # hot_start=True
                    )  # see help(egor) for options

# Restrict regression and correlation models used
#egor = egx.Egor(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [5]:
res = egor.minimize(n_eval=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.egor:Compute initial LHS with 10 points
INFO:egobox_ego.egor:Objective initial clustering...
INFO:egobox_moe.algorithm:Best expert Constant_Matern52 accuracy=0.0000028275355870244213
INFO:egobox_ego.egor:... Best nb of clusters / mixture for Objective: 1 / Mixture[Smooth(1)]
INFO:egobox_ego.egor:Constraint[1] reclustering... initial clustering...
INFO:egobox_moe.algorithm:Best expert Constant_Matern32 accuracy=1.7940404074199112
INFO:egobox_ego.egor:... Best nb of clusters / mixture for Constraint[1] reclustering...: 1 / Mixture[Smooth(1)]
INFO:egobox_ego.egor:Constraint[2] reclustering... initial clustering...
INFO:egobox_moe.algorithm:Best expert Constant_Matern52 accuracy=3.2830674611703543
INFO:egobox_ego.egor:... Best nb of clusters / mixture for Constraint[2] reclustering...: 1 / Mixture[Smooth(1)]
INFO:egobox_ego.egor:+1 point(s), total : 11 points
INFO:egobox_ego.egor:End iteration 1/20: Best fun(x)=[-5.364787037994645, -0.40441301551314135, -0.03911667428052823

Optimization f=[-5.50872107014391, 0.00021118534990627325, 0.0009082820234311839] at [2.329574380329236, 3.179146689814674]


## Mixed integer test function

In [7]:
xspecs_mixint_xsinx = [egx.Vspec(egx.Vtype(egx.Vtype.INT), [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

In [8]:
egor = egx.Egor(mixint_xsinx, xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     expected=egx.ExpectedOptimum(val=-15.12, tol=1e-2),
                    )  # see help(egor) for options
res = egor.minimize(n_eval=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.egor:Compute initial LHS with 3 points
INFO:egobox_ego.egor:Objective initial clustering...
INFO:egobox_moe.algorithm:Best expert Constant_SquaredExponential accuracy=3.7451341329849517
INFO:egobox_ego.egor:... Best nb of clusters / mixture for Objective: 1 / Mixture[Smooth(1)]
INFO:egobox_ego.egor:+1 point(s), total : 4 points
INFO:egobox_ego.egor:End iteration 1/27: Best fun(x)=[-2.092831226407067] at x=[14.02984666810414]
INFO:egobox_moe.algorithm:Best expert Constant_AbsoluteExponential accuracy=3.3168146194905512
INFO:egobox_ego.egor:+1 point(s), total : 5 points
INFO:egobox_ego.egor:End iteration 2/27: Best fun(x)=[-5.703849598284653] at x=[15.357351952292428]
INFO:egobox_moe.algorithm:Best expert Constant_SquaredExponential accuracy=2.7128336667262167
INFO:egobox_ego.egor:+1 point(s), total : 6 points
INFO:egobox_ego.egor:End iteration 3/27: Best fun(x)=[-9.285318589373164] at x=[15.847656090311458]
INFO:egobox_moe.algorithm:Best expert Constant_AbsoluteExponenti

Optimization f=[-15.121611536143384] at [19.0]


## Usage

In [9]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(fun, n_cstr=0, cstr_tol=1e-06, n_start=20, n_doe=0, regression_spec=7, correlation_spec=15, infill_strategy=1, q_parallel=1, par_infill_strategy=1, infill_optimizer=1, n_clusters=1)
 |  
 |  Optimizer constructor
 |  
 |  fun: array[n, nx]) -> array[n, ny]
 |       the function to be minimized
 |       fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |          obj is the objective function [n, nx] -> [n, 1]
 |          cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |          an k the number of constraints (n_cstr)
 |          hence ny = 1 (obj) + k (cstrs)
 |       cstr functions are expected be negative (<=0) at the optimum.
 |  
 |   n_cstr (int):
 |       the number of constraint functions.
 |  
 |   cstr_tol (float):
 |       tolerance on constraints violation (cstr < tol).
 |  
 |   xspecs (list(Vspec)) where Vspec(vtype=FLOAT|INT, vlimits=[lower bound, upper bound]):
 |       Bounds of the nx compo